In [2]:
import socket

In [3]:
IP_ADDRESS = "192.168.0.17"
TCP_Port = 23
UDP_Port = 4660

In [15]:
readBytes = bytearray([0xFF,0xc0,0x00,0xFF]) + 0x1b5.to_bytes(4,'big')
print(readBytes)

bytearray(b'\xff\xc0\x00\xff\x00\x00\x01\xb5')


In [5]:
sock = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)

In [16]:
sock.sendto(readBytes,(IP_ADDRESS,UDP_Port))
rec = sock.recvfrom(1024)
print(rec)

(b'\xff\xc8\x00\xff\x00\x00\x01\xb5\x00\x80\x00\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xea`\x13\x88+\xf2\x01\xf4\x13\x87\t\x7f\x7f\x7f\x7f\x7f\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8', ('192.168.0.1

In [17]:
rec[0]

b'\xff\xc8\x00\xff\x00\x00\x01\xb5\x00\x80\x00\x00\xff\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xea`\x13\x88+\xf2\x01\xf4\x13\x87\t\x7f\x7f\x7f\x7f\x7f\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8\x00\x11\x00\x17\x05\xb4\x124\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03\xe8\xea`\x13\x88+\xf2\x01\xf4\x01\xf4\xc0\xa8'

In [12]:
bin(0x8)

'0b1000'